In [1]:
access_token = 'EAAH24PQU4egBACZB6zHQmKZB9WfPR1LVudZArgsjLso7xtvG4ZCx9w77RNCkHo4Cz3nRj7HePcrCYEWXknm4ZBMoSm20Tnjp0ZCRNYINSjw2raSLPFfD3coNEz9LO6GIZA3dFBQdi5ndZBv6WPNBpN9UeBr2LS3iBmZBGsEYWqwzDR7OxJiL2GUDZAL3j7zOllyMcZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')
!pip install facebook-business

import pandas as pd
import numpy as np

# pd.options.mode.chained_assignment = None  # default='warn'

#from sqlalchemy import create_engine

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adactivity import AdActivity
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from tqdm import tqdm

import psycopg2
from psycopg2 import OperationalError

import json
import datetime

from datetime import timedelta, datetime, date, time
import time



def list_days(start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    d = timedelta(1)
    date_list = []
    while start_date != end_date + d:
        date_list.append(start_date)
        start_date = start_date + d

    return date_list

status = lambda x: True if len(x) > 0 else False

def store_actions(actions, table):
    for act in actions:
        fields = ','.join(act.keys())
        values = ','.join(map(lambda x:f"%({x})s",act.keys()))
        query =   f"INSERT INTO {table} ({fields}) values ({values})"
        cur.execute(query, act)
    
def log_actions(account_id, day, actions, table_log):
    log = (account_id, day, status(actions), 
                                    datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
    query =   f"INSERT INTO {table_log} values {log}"
    cur.execute(query)
    
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        connection.cursor().execute('SELECT 1')
    return connection.cursor(), connection
 

In [3]:
print('start processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start_date = (datetime.utcnow() - timedelta(days = 30)).strftime('%Y-%m-%d')
yesterday = (datetime.utcnow() - timedelta(days = 1)).strftime('%Y-%m-%d')

try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

#engine = create_engine(address)

FacebookAdsApi.init(access_token=access_token)

me = User(fbid='me')
id_list = list(me.get_ad_accounts())
id_list = list(map(lambda x:x.export_all_data(), id_list))
id_list = list(map(lambda x:x['account_id'], id_list))

account_id_list = id_list
# account_id_list = []
# for account_id in id_list :
#     my_account = AdAccount('act_' + account_id)
#     try:
#         my_account.remote_read(fields=[AdAccount.Field.account_id])
#         account_id_list.append(account_id)
#     except FacebookRequestError:
#         print('missing permissions ' + account_id)

fields_activity = list(set(AdActivity.Field.__dict__.keys()) - set(['__module__','__doc__']))

table = 'actions'
table_log = 'actions_log'

cur.execute(f"""CREATE TABLE if not exists {table_log}(
    id VARCHAR(50),     date date,     with_data BOOLEAN,
    recording_date timestamp without time zone,
    PRIMARY KEY (id, date))""")

cur.execute(f"""CREATE UNIQUE INDEX if not exists {table_log}_idx ON {table_log} (id, date)""")

cur.execute(f"""CREATE TABLE if not exists {table}(
     account_id VARCHAR(50),      date date,         actor_id VARCHAR(100),
     actor_name VARCHAR(100),     application_id VARCHAR(100),      application_name VARCHAR(100),
     date_time_in_timezone timestamp,      event_time timestamp,      event_type VARCHAR(100),
     extra_data JSON,      object_id VARCHAR(100),      object_name VARCHAR(200),
     object_type VARCHAR(100),      translated_event_type VARCHAR(100),      id VARCHAR(100),
     FOREIGN KEY (account_id, date) REFERENCES {table_log}(id, date) ON UPDATE CASCADE)""")

connection.commit()

print('start uploading \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))



for account_id in tqdm(account_id_list):
    account = AdAccount('act_' + account_id)

    days = tuple(map(lambda x:x.strftime('%Y-%m-%d'), list_days(start_date, yesterday)))

    query = """SELECT date FROM actions_log where date in %s and id = %s""" 
    days_in_db = pd.read_sql(query, connection, params = (days, account_id)).date
    days_in_db = set(map(lambda x:x.strftime('%Y-%m-%d'), days_in_db))

    days = set(days) - days_in_db

    for day in days:

        since = day
        until = (datetime.strptime(day, '%Y-%m-%d') + timedelta(days = 1)).strftime('%Y-%m-%d')
        params = {'since': since, 'until': until}

        try:
            raw_data = account.get_activities(fields= fields_activity,params = params)
            actions = list(map(lambda x:x.export_all_data(),raw_data))
            tuple(map(lambda x:x.update(dict(account_id=account_id)),actions))
            tuple(map(lambda x:x.update(dict(date=day)),actions))
            actions = tuple(actions)
            cur, connection = reconnection_to_database(host, port, database, user, password, connection)
            log_actions(account_id, day, actions, table_log)                
            store_actions(actions, table)
            connection.commit()

        except FacebookRequestError:
            print('error with ' + account_id + ' and day ' + day)

        except FacebookBadObjectError:
            print('bad data to set object data with ' + account_id + ' and day ' + day)

#         except OperationalError:
#             t = 60
#             print(f'OperationalError, sleeping {t} seconds')
#             time.sleep(t)
#             cur, connection = connection_to_database(host, port, database, user, password)
#             print(f'again in progress')

print('end processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start processing 	2020-07-13 14:12:49


  0%|          | 0/21 [00:00<?, ?it/s]

start uploading 	2020-07-13 14:12:50


100%|██████████| 21/21 [10:28<00:00, 29.92s/it]

end processing 	2020-07-13 14:23:18
